# Evaluate the box below to initialize the web UI.

In [ ]:
import os
import pprint
import ipywidgets as widgets
import ray

from IPython.display import display
from ipywidgets import interact

In [ ]:
redis_address = os.environ["REDIS_ADDRESS"]
ray.init(redis_address=redis_address)

In [ ]:
pp = pprint.PrettyPrinter(indent=3)
task_table = ray.global_state.task_table()
task_profiles = ray.global_state.task_profiles()
task_search = widgets.Text(
    value="",
    placeholder='Task ID',
    description='Search for a task:',
    disabled=False
)
display(task_search)

def handle_submit(sender):
    if task_search.value in task_table and task_search.value in task_profiles: 
        task_info = dict(list(task_table[task_search.value].items()) + list(task_profiles[task_search.value].items()))
        pp.pprint(task_info)
    else:
        print("Task not found.")
        
task_search.on_submit(handle_submit)